In [1]:
%cd ..

/home/eli/AnacondaProjects/ppc_experiments


In [2]:
import argparse
import collections
import numpy as np
import pyro
import torch
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
import trainer.trainer as module_trainer

In [3]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [4]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [5]:
from utils import read_json

config = read_json("experiments/ppc_mnist_config.json")
config = ConfigParser(config)

In [6]:
logger = config.get_logger('train')

# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

# build model architecture, then print to console
model = config.init_obj('arch', module_arch)
logger.info(model)

# get function handles of metrics
metrics = [getattr(module_metric, met) for met in config['metrics']]

# build optimizer.
if "lr_scheduler" in config:
    lr_scheduler = getattr(pyro.optim, config["lr_scheduler"]["type"])
    lr_scheduler = optimizer = lr_scheduler({
        "optimizer": getattr(torch.optim, config["optimizer"]["type"]),
        "optim_args": config["optimizer"]["args"]["optim_args"],
        **config["lr_scheduler"]["args"]
    })
else:
    optimizer = config.init_obj('optimizer', pyro.optim)
    lr_scheduler = None

# build trainer
# kwargs = config['trainer'].pop('args')
trainer = config.init_obj('trainer', module_trainer, model, metrics, optimizer,
                          config=config, data_loader=data_loader,
                          valid_data_loader=valid_data_loader,
                          lr_scheduler=lr_scheduler)

MnistPpc(
  (prior): GaussianPrior()
  (decoder1): ConditionalGaussian(
    (decoder): Sequential(
      (0): ReLU()
      (1): Linear(in_features=20, out_features=128, bias=True)
    )
  )
  (decoder2): ConditionalGaussian(
    (decoder): Sequential(
      (0): ReLU()
      (1): Linear(in_features=128, out_features=256, bias=True)
    )
  )
  (likelihood): MlpBernoulliLikelihood(
    (decoder): Sequential(
      (0): ReLU()
      (1): Linear(in_features=256, out_features=784, bias=True)
    )
  )
  (graph): PpcGraphicalModel()
)
Trainable parameters: 319540
Initialize particles: train batch 0
Initialize particles: train batch 1
Initialize particles: train batch 2
Initialize particles: train batch 3
Initialize particles: train batch 4
Initialize particles: train batch 5
Initialize particles: train batch 6
Initialize particles: train batch 7
Initialize particles: train batch 8
Initialize particles: train batch 9
Initialize particles: train batch 10
Initialize particles: train batch 11
I

In [7]:
logger.info(trainer.config.log_dir)

saved/log/Mnist_Ppc/0327_194126


In [ ]:
trainer.train()

Train Epoch: 1 [0/54000 (0%)] Loss: 767.783508
Train Epoch: 1 [4096/54000 (8%)] Loss: -504.366394
Train Epoch: 1 [8192/54000 (15%)] Loss: -561.750488
Train Epoch: 1 [12288/54000 (23%)] Loss: -597.216248
Train Epoch: 1 [16384/54000 (30%)] Loss: -603.443420
Train Epoch: 1 [20480/54000 (38%)] Loss: -641.130493
Train Epoch: 1 [24576/54000 (46%)] Loss: -645.749573
Train Epoch: 1 [28672/54000 (53%)] Loss: -702.476318
Train Epoch: 1 [32768/54000 (61%)] Loss: -710.776550
Train Epoch: 1 [36864/54000 (68%)] Loss: -727.786621
Train Epoch: 1 [40960/54000 (76%)] Loss: -759.938354
Train Epoch: 1 [45056/54000 (83%)] Loss: -793.577148
Train Epoch: 1 [49152/54000 (91%)] Loss: -793.875183
Train Epoch: 1 [53248/54000 (99%)] Loss: -828.922119
    epoch          : 1
    loss           : -647.0783520468038
    ess            : 3.8803834474482244
    log_marginal   : 647.1240250646221
    val_loss       : -805.2631200154623
    val_ess        : 3.8806121249993644
    val_log_marginal: 805.3075307210287
Train

In [ ]:
trainer.model.eval()
trainer.model.cpu()
trainer.cpu()
trainer.train_particles.cpu()
trainer.valid_particles.cpu()

In [ ]:
trainer.model.graph.clear()
trainer._load_particles(range(trainer.data_loader.batch_size), False)

In [ ]:
for site in trainer.model.graph.nodes:
    trainer.model.graph.nodes[site]['is_observed'] = trainer.model.graph.nodes[site]['value'] is not None

In [ ]:
import utils

In [ ]:
with pyro.plate_stack("forward", (trainer.num_particles, trainer.data_loader.batch_size)):
    model = pyro.condition(trainer.model, data={k: v['value'] for k, v in trainer.model.graph.nodes.items()})
    xs = model()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for i in range(10):
    plt.imshow(xs.mean(dim=0)[i].squeeze().detach().cpu().numpy())
    plt.show()